In [1]:
# Import modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(
    Path("alzheimer_clean.csv")   
)

# Review the DataFrame
df.head()

,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,SleepQuality,...,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis
0,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,9.025679,...,6.518877,0,0,1.725883,0,0,0,1,0,0
1,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,7.151293,...,7.118696,0,0,2.592424,0,0,0,0,1,0
2,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,9.673574,...,5.895077,0,0,7.119548,0,1,0,1,0,0
3,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,8.392554,...,8.965106,0,1,6.481226,0,0,0,0,0,0
4,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,5.597238,...,6.045039,0,0,0.014691,0,0,1,1,0,0


In [3]:
o5_df = df[["Diagnosis", "FunctionalAssessment", "ADL", "MemoryComplaints", "MMSE", "BehavioralProblems"]]

In [4]:
y = o5_df["Diagnosis"]
x = o5_df.drop(columns="Diagnosis")

In [5]:
# Encode the categorical variables using get_dummies
X = pd.get_dummies(x)

In [6]:
# split original dataset X and y into a training set + temp set (test_size=0.2 for 20% of data going towards the temp set and 80% for training set)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=1)

# split temp set into validation set + test set (50/50) 
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1)

# goal = train using 80% of original data, tune hyperparameters using the validation set + evaluate model performance on unseen data using test data

In [7]:
print(X_train.shape, X_val.shape, X_test.shape)

(1719, 5) (215, 5) (215, 5)


In [8]:
# Instantiate a StandardScaler instance
scaler = StandardScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled = X_scaler.transform(X_test)

# Transform the validation data using the scaler
X_val_scaled = X_scaler.transform(X_val)

Doing a Grid Search for n_neighours optimal value using cross-validation in Python with SciKit-Learn

In [9]:
# Define the parameter grid with the correct parameter name 'n_neighbors'
param_grid = {'n_neighbors': [3, 5, 7, 9, 11]}

# Create a KNN classifier
knn = KNeighborsClassifier()

# Perform grid search with cross-validation
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Access the best parameters and print the best n_neighbors value
best_n_neighbors = grid_search.best_params_['n_neighbors']
print("Best n_neighbors value:", best_n_neighbors)

Best n_neighbors value: 3


Trying the "rule of thumb for n_neighbours" = square root of total number of samples in the dataset (1719 for training set = 41)

In [10]:
# Define the parameter grid with the correct parameter name 'n_neighbors'
param_grid = {'n_neighbors': [35, 37, 39, 41, 43]}

# Create a KNN classifier
knn = KNeighborsClassifier()

# Perform grid search with cross-validation
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Access the best parameters and print the best n_neighbors value
best_n_neighbors = grid_search.best_params_['n_neighbors']
print("Best n_neighbors value:", best_n_neighbors)

Best n_neighbors value: 35


try n_neighbors = 3

In [11]:
# Import the KNeighborsClassifier module from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Instantiate the KNeighborsClassifier model with n_neighbors = 19 
knn = KNeighborsClassifier(n_neighbors=3)

In [12]:
# Train the model using the training data
knn.fit(X_train_scaled, y_train)


KNeighborsClassifier(n_neighbors=3)

In [13]:
validation_accuracy = knn.score(X_val, y_val)
print("Validation Accuracy:", validation_accuracy)

Validation Accuracy: 0.6651162790697674


In [14]:
# Create predictions using the testing data
y_pred = knn.predict(X_test_scaled)

In [15]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.89      0.90       139
           1       0.81      0.82      0.81        76

    accuracy                           0.87       215
   macro avg       0.85      0.85      0.85       215
weighted avg       0.87      0.87      0.87       215



Try n_neighbors = 35

---> lowered validation and accuracy so keep n_neighbours = 3

In [16]:
# Import the KNeighborsClassifier module from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Instantiate the KNeighborsClassifier model with n_neighbors = 19 
knn = KNeighborsClassifier(n_neighbors=35)

In [17]:
# Train the model using the training data
knn.fit(X_train_scaled, y_train)


KNeighborsClassifier(n_neighbors=35)

In [18]:
validation_accuracy = knn.score(X_val, y_val)
print("Validation Accuracy:", validation_accuracy)

Validation Accuracy: 0.641860465116279


In [19]:
# Create predictions using the testing data
y_pred = knn.predict(X_test_scaled)

In [20]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89       139
           1       0.80      0.80      0.80        76

    accuracy                           0.86       215
   macro avg       0.85      0.85      0.85       215
weighted avg       0.86      0.86      0.86       215



Normalization to scale features to a range between 0 and 1 (want to bring data to common scale)

---> decreased accuracy, recall, precision. Keep standardscaler 

In [21]:
from sklearn.preprocessing import MinMaxScaler

In [22]:
# Create a MinMaxScaler instance
scaler = MinMaxScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled_norm = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled_norm = X_scaler.transform(X_test)

# Transform the validation data using the scaler
X_val_scaled_norm = X_scaler.transform(X_val)

In [23]:
from sklearn.pipeline import make_pipeline

# Create a pipeline with StandardScaler and KNN classifier
pipeline = make_pipeline(MinMaxScaler(), KNeighborsClassifier(n_neighbors=3))

# Fit the pipeline on the training data and make predictions
pipeline.fit(X_train, y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('kneighborsclassifier', KNeighborsClassifier(n_neighbors=3))])

In [24]:
# Instantiate the KNeighborsClassifier model with n_neighbors = 19 
knn = KNeighborsClassifier(n_neighbors=3)

In [25]:
# Train the model using the training data
knn.fit(X_train_scaled_norm, y_train)

KNeighborsClassifier(n_neighbors=3)

In [26]:
validation_accuracy = knn.score(X_val, y_val)
print("Validation Accuracy:", validation_accuracy)

Validation Accuracy: 0.6511627906976745


In [27]:
# Create predictions using the testing data
y_pred = knn.predict(X_test)

In [28]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.64      0.98      0.78       139
           1       0.25      0.01      0.03        76

    accuracy                           0.64       215
   macro avg       0.45      0.50      0.40       215
weighted avg       0.51      0.64      0.51       215



Normalization and Pipeline Feature has the same validation accuracy, but lowers f1-score whilst increasing recall for [0], which decreasing for [1]. 

NO IMPROVEMENT WITH CHANGE IN WEIGHT BY DISTANCE (uniform and distance the same)

In [29]:
# Create a k-NN classifier with distance-based weighting
knn_distance = KNeighborsClassifier(n_neighbors=3, weights='distance')

In [30]:
# Train the model using the training data
knn_distance.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=3, weights='distance')

In [31]:
validation_accuracy = knn_distance.score(X_val, y_val)
print("Validation Accuracy:", validation_accuracy)

# validation decreased with weight change

Validation Accuracy: 0.6651162790697674


In [32]:
# Create predictions using the testing data
y_pred = knn_distance.predict(X_test_scaled)

In [33]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.89      0.90       139
           1       0.81      0.82      0.81        76

    accuracy                           0.87       215
   macro avg       0.85      0.85      0.85       215
weighted avg       0.87      0.87      0.87       215



Try lowered n_neighbors (=19)

In [34]:
# Import the KNeighborsClassifier module from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Instantiate the KNeighborsClassifier model with n_neighbors = 19 
knn = KNeighborsClassifier(n_neighbors=19)

In [35]:
# Train the model using the training data
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=19)

In [36]:
validation_accuracy = knn.score(X_val, y_val)
print("Validation Accuracy:", validation_accuracy)

Validation Accuracy: 0.641860465116279


In [37]:
# Create predictions using the testing data
y_pred = knn.predict(X_test_scaled)

In [38]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.91      0.90       139
           1       0.83      0.82      0.82        76

    accuracy                           0.87       215
   macro avg       0.86      0.86      0.86       215
weighted avg       0.87      0.87      0.87       215



TESTING DEEP LEARNING MODELS

In [39]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=5))

# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

# results promising; but attempt to improve 

Epoch 1/50
54/54 [==============================] - 1s 4ms/step - loss: 0.8073 - accuracy: 0.3775 - val_loss: 0.7624 - val_accuracy: 0.4047
Epoch 2/50
54/54 [==============================] - 0s 1ms/step - loss: 0.6742 - accuracy: 0.5777 - val_loss: 0.6539 - val_accuracy: 0.6372
Epoch 3/50
54/54 [==============================] - 0s 963us/step - loss: 0.5842 - accuracy: 0.7545 - val_loss: 0.5780 - val_accuracy: 0.7256
Epoch 4/50
54/54 [==============================] - 0s 1ms/step - loss: 0.5207 - accuracy: 0.7999 - val_loss: 0.5253 - val_accuracy: 0.7628
Epoch 5/50
54/54 [==============================] - 0s 1ms/step - loss: 0.4752 - accuracy: 0.8220 - val_loss: 0.4853 - val_accuracy: 0.7860
Epoch 6/50
54/54 [==============================] - 0s 959us/step - loss: 0.4416 - accuracy: 0.8371 - val_loss: 0.4535 - val_accuracy: 0.7907
Epoch 7/50
54/54 [==============================] - 0s 1ms/step - loss: 0.4159 - accuracy: 0.8482 - val_loss: 0.4277 - val_accuracy: 0.8047
Epoch 8/50
54/54

First start w/ grid search to see if random neuron number was optimal

In [40]:
# Define the deep learning model function

def create_model(neurons=16):
    nn_model = tf.keras.models.Sequential()
    nn_model.add(tf.keras.layers.Dense(units=neurons, activation="relu", input_dim=5))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return nn_model

# Create a KerasClassifier based on the model function
nn_model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=10, verbose=0)

# Define the grid of hyperparameters to search
param_grid = {'neurons': [8, 16, 32, 64]}

# Perform grid search
grid = GridSearchCV(estimator=nn_model, param_grid=param_grid, cv=5)
grid_result = grid.fit(X_train_scaled, y_train)

# Get the best parameters and results
print("Best Parameters: ", grid_result.best_params_)
print("Best Score: ", grid_result.best_score_)

Best Parameters:  {'neurons': 64}
Best Score:  0.9197149038314819


Test model using 64 neurons in layer

-----> optimized 8 neurons x 1 layer (50 epochs to test)

In [41]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=64, activation="relu", input_dim=5))

# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Epoch 1/50
54/54 [==============================] - 1s 4ms/step - loss: 0.6135 - accuracy: 0.6958 - val_loss: 0.5050 - val_accuracy: 0.8093
Epoch 2/50
54/54 [==============================] - 0s 1ms/step - loss: 0.4671 - accuracy: 0.8400 - val_loss: 0.4092 - val_accuracy: 0.8465
Epoch 3/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3961 - accuracy: 0.8586 - val_loss: 0.3636 - val_accuracy: 0.8512
Epoch 4/50
54/54 [==============================] - 0s 2ms/step - loss: 0.3628 - accuracy: 0.8697 - val_loss: 0.3403 - val_accuracy: 0.8465
Epoch 5/50
54/54 [==============================] - 0s 2ms/step - loss: 0.3464 - accuracy: 0.8720 - val_loss: 0.3276 - val_accuracy: 0.8605
Epoch 6/50
54/54 [==============================] - 0s 3ms/step - loss: 0.3369 - accuracy: 0.8749 - val_loss: 0.3194 - val_accuracy: 0.8605
Epoch 7/50
54/54 [==============================] - ETA: 0s - loss: 0.3269 - accuracy: 0.87 - 0s 2ms/step - loss: 0.3310 - accuracy: 0.8767 - val_loss: 0.3138 -

Increase neurons to = 64 increase accuracy for model

Checking optimization of epochs # using GridSearchCV

In [42]:
# Define the deep learning model function

def create_model(neurons=64):
    nn_model = tf.keras.models.Sequential()
    nn_model.add(tf.keras.layers.Dense(units=neurons, activation="relu", input_dim=5))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return nn_model

# Create a KerasClassifier based on the model function
nn_model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=10, verbose=0)

# Define the grid of hyperparameters to search
param_grid = {'epochs': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]}

# Perform grid search
grid = GridSearchCV(estimator=nn_model, param_grid=param_grid, cv=5)
grid_result = grid.fit(X_train_scaled, y_train)

# Get the best parameters and results
print("Best Parameters: ", grid_result.best_params_)
print("Best Score: ", grid_result.best_score_)

Best Parameters:  {'epochs': 100}
Best Score:  0.9371686100959777


In [43]:
# Define the deep learning model function

def create_model(neurons=64):
    nn_model = tf.keras.models.Sequential()
    nn_model.add(tf.keras.layers.Dense(units=neurons, activation="relu", input_dim=5))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return nn_model

# Create a KerasClassifier based on the model function
nn_model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=10, verbose=0)

# Define the grid of hyperparameters to search
param_grid = {'epochs': [100, 150, 200]}

# Perform grid search
grid = GridSearchCV(estimator=nn_model, param_grid=param_grid, cv=5)
grid_result = grid.fit(X_train_scaled, y_train)

# Get the best parameters and results
print("Best Parameters: ", grid_result.best_params_)
print("Best Score: ", grid_result.best_score_)

Best Parameters:  {'epochs': 200}
Best Score:  0.9412349939346314


In [44]:
# Define the deep learning model function

def create_model(neurons=64):
    nn_model = tf.keras.models.Sequential()
    nn_model.add(tf.keras.layers.Dense(units=neurons, activation="relu", input_dim=5))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return nn_model

# Create a KerasClassifier based on the model function
nn_model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=10, verbose=0)

# Define the grid of hyperparameters to search
param_grid = {'epochs': [200, 300, 400]}

# Perform grid search
grid = GridSearchCV(estimator=nn_model, param_grid=param_grid, cv=5)
grid_result = grid.fit(X_train_scaled, y_train)

# Get the best parameters and results
print("Best Parameters: ", grid_result.best_params_)
print("Best Score: ", grid_result.best_score_)

Best Parameters:  {'epochs': 300}
Best Score:  0.9412417650222779


Keeping epochs=200 because lowest runtime and highest score

Checking validation set using new parameters (optimized epochs=200 as well as neuron =8 in single layer)

---> lower than for epochs = 50, so try epochs=100

---> epochs = 100 also has lower results than epochs=50. Keep epochs = 50

In [45]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=5))

# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=200, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Epoch 1/200
54/54 [==============================] - 1s 4ms/step - loss: 0.6406 - accuracy: 0.6300 - val_loss: 0.6078 - val_accuracy: 0.6419
Epoch 2/200
54/54 [==============================] - 0s 1ms/step - loss: 0.5926 - accuracy: 0.6742 - val_loss: 0.5688 - val_accuracy: 0.6651
Epoch 3/200
54/54 [==============================] - 0s 1ms/step - loss: 0.5541 - accuracy: 0.7062 - val_loss: 0.5348 - val_accuracy: 0.6977
Epoch 4/200
54/54 [==============================] - 0s 1ms/step - loss: 0.5204 - accuracy: 0.7464 - val_loss: 0.5035 - val_accuracy: 0.7349
Epoch 5/200
54/54 [==============================] - 0s 1ms/step - loss: 0.4882 - accuracy: 0.7853 - val_loss: 0.4713 - val_accuracy: 0.7674
Epoch 6/200
54/54 [==============================] - 0s 1ms/step - loss: 0.4566 - accuracy: 0.8185 - val_loss: 0.4410 - val_accuracy: 0.8000
Epoch 7/200
54/54 [==============================] - 0s 1ms/step - loss: 0.4276 - accuracy: 0.8412 - val_loss: 0.4124 - val_accuracy: 0.8279
Epoch 8/200
5

In [46]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=5))

# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Epoch 1/100
54/54 [==============================] - 1s 4ms/step - loss: 0.8214 - accuracy: 0.5457 - val_loss: 0.6950 - val_accuracy: 0.6419
Epoch 2/100
54/54 [==============================] - 0s 1ms/step - loss: 0.7298 - accuracy: 0.6062 - val_loss: 0.6337 - val_accuracy: 0.6930
Epoch 3/100
54/54 [==============================] - 0s 1ms/step - loss: 0.6578 - accuracy: 0.6475 - val_loss: 0.5836 - val_accuracy: 0.7209
Epoch 4/100
54/54 [==============================] - 0s 2ms/step - loss: 0.5969 - accuracy: 0.6992 - val_loss: 0.5393 - val_accuracy: 0.7721
Epoch 5/100
54/54 [==============================] - 0s 1ms/step - loss: 0.5453 - accuracy: 0.7405 - val_loss: 0.5029 - val_accuracy: 0.7674
Epoch 6/100
54/54 [==============================] - 0s 1ms/step - loss: 0.5031 - accuracy: 0.7789 - val_loss: 0.4707 - val_accuracy: 0.8000
Epoch 7/100
54/54 [==============================] - 0s 1ms/step - loss: 0.4690 - accuracy: 0.8133 - val_loss: 0.4457 - val_accuracy: 0.8140
Epoch 8/100
5

Checking parameters neurons x optimizer for optimization

In [47]:
# Define the deep learning model function

def create_model(neurons=8, optimizer="adam"):
    nn_model = tf.keras.models.Sequential()
    nn_model.add(tf.keras.layers.Dense(units=neurons, activation="relu", input_dim=5))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    nn_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return nn_model

# Create a KerasClassifier based on the model function
nn_model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=10, verbose=0)

# Define the grid of hyperparameters to search
param_grid = {'neurons': [8, 16, 32],'optimizer': ["adam", "SGD", "RMSprop"]}

# Perform grid search
grid = GridSearchCV(estimator=nn_model, param_grid=param_grid, cv=5)
grid_result = grid.fit(X_train_scaled, y_train)

# Get the best parameters and results
print("Best Parameters: ", grid_result.best_params_)
print("Best Score: ", grid_result.best_score_)

Best Parameters:  {'neurons': 32, 'optimizer': 'adam'}
Best Score:  0.9063478946685791


In [48]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=64, activation="relu", input_dim=5))

# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="RMSprop", metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Epoch 1/50
54/54 [==============================] - 1s 3ms/step - loss: 0.5528 - accuracy: 0.7993 - val_loss: 0.4766 - val_accuracy: 0.8186
Epoch 2/50
54/54 [==============================] - 0s 1ms/step - loss: 0.4351 - accuracy: 0.8505 - val_loss: 0.3932 - val_accuracy: 0.8326
Epoch 3/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3794 - accuracy: 0.8610 - val_loss: 0.3493 - val_accuracy: 0.8465
Epoch 4/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3548 - accuracy: 0.8668 - val_loss: 0.3294 - val_accuracy: 0.8465
Epoch 5/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3423 - accuracy: 0.8691 - val_loss: 0.3158 - val_accuracy: 0.8605
Epoch 6/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3341 - accuracy: 0.8749 - val_loss: 0.3113 - val_accuracy: 0.8558
Epoch 7/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3287 - accuracy: 0.8743 - val_loss: 0.3049 - val_accuracy: 0.8605
Epoch 8/50
54/54 [==

neuron =64, epochs =50, optimizer = RMSprop gives the best balance between test accuracy vs. validation accuracy

Testing other activation functions

---> AF = tanh decreases accuracy

---> PReLU also increases accuracy + validation

In [49]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=64, activation="tanh", input_dim=5))

# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="RMSprop", metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Epoch 1/50
54/54 [==============================] - 1s 4ms/step - loss: 0.5328 - accuracy: 0.7539 - val_loss: 0.4408 - val_accuracy: 0.8047
Epoch 2/50
54/54 [==============================] - 0s 1ms/step - loss: 0.4069 - accuracy: 0.8470 - val_loss: 0.3846 - val_accuracy: 0.8140
Epoch 3/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3766 - accuracy: 0.8534 - val_loss: 0.3718 - val_accuracy: 0.8140
Epoch 4/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3695 - accuracy: 0.8534 - val_loss: 0.3637 - val_accuracy: 0.8186
Epoch 5/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3676 - accuracy: 0.8505 - val_loss: 0.3626 - val_accuracy: 0.8186
Epoch 6/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3666 - accuracy: 0.8534 - val_loss: 0.3614 - val_accuracy: 0.8233
Epoch 7/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3656 - accuracy: 0.8540 - val_loss: 0.3599 - val_accuracy: 0.8186
Epoch 8/50
54/54 [==

In [50]:
from tensorflow.keras.layers import PReLU

# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=64, input_dim=5))
nn_model.add(PReLU())  # Adding PReLU activation function here

# Output layer for binary classification (1 neuron for 0 or 1)
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="RMSprop", metrics=["accuracy"])

# Train the model with validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data and validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Epoch 1/50
54/54 [==============================] - 1s 4ms/step - loss: 0.5554 - accuracy: 0.7749 - val_loss: 0.4676 - val_accuracy: 0.8279
Epoch 2/50
54/54 [==============================] - 0s 1ms/step - loss: 0.4225 - accuracy: 0.8581 - val_loss: 0.3805 - val_accuracy: 0.8372
Epoch 3/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3639 - accuracy: 0.8662 - val_loss: 0.3396 - val_accuracy: 0.8326
Epoch 4/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3414 - accuracy: 0.8674 - val_loss: 0.3223 - val_accuracy: 0.8419
Epoch 5/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3304 - accuracy: 0.8732 - val_loss: 0.3125 - val_accuracy: 0.8465
Epoch 6/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3237 - accuracy: 0.8755 - val_loss: 0.3045 - val_accuracy: 0.8558
Epoch 7/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3184 - accuracy: 0.8784 - val_loss: 0.3042 - val_accuracy: 0.8558
Epoch 8/50
54/54 [==

Testing addition of more hidden layers

---> adding one hidden layer increased accuracy and decreased loss

---> adding two hidden layers increased accuracy and decreased loss

---> adding three hidden layers decreased accuracy and increased loss

In [51]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=64, input_dim=5))
nn_model.add(tf.keras.layers.Dense(units=64))

nn_model.add(PReLU())  # Adding PReLU activation function here
# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="RMSprop", metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Epoch 1/50
54/54 [==============================] - 1s 4ms/step - loss: 0.4349 - accuracy: 0.8464 - val_loss: 0.3411 - val_accuracy: 0.8558
Epoch 2/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3389 - accuracy: 0.8720 - val_loss: 0.3164 - val_accuracy: 0.8558
Epoch 3/50
54/54 [==============================] - 0s 2ms/step - loss: 0.3236 - accuracy: 0.8813 - val_loss: 0.3051 - val_accuracy: 0.8791
Epoch 4/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3138 - accuracy: 0.8837 - val_loss: 0.2982 - val_accuracy: 0.8837
Epoch 5/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3067 - accuracy: 0.8854 - val_loss: 0.2883 - val_accuracy: 0.8837
Epoch 6/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3006 - accuracy: 0.8889 - val_loss: 0.2851 - val_accuracy: 0.8884
Epoch 7/50
54/54 [==============================] - 0s 1ms/step - loss: 0.2967 - accuracy: 0.8912 - val_loss: 0.2800 - val_accuracy: 0.8930
Epoch 8/50
54/54 [==

In [52]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=64, input_dim=5))
nn_model.add(tf.keras.layers.Dense(units=64))
nn_model.add(tf.keras.layers.Dense(units=64))

nn_model.add(PReLU())  # Adding PReLU activation function here
# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="RMSprop", metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Epoch 1/50
54/54 [==============================] - 1s 5ms/step - loss: 0.3922 - accuracy: 0.8429 - val_loss: 0.3044 - val_accuracy: 0.8605
Epoch 2/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3216 - accuracy: 0.8755 - val_loss: 0.2892 - val_accuracy: 0.8791
Epoch 3/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3107 - accuracy: 0.8743 - val_loss: 0.2980 - val_accuracy: 0.8744
Epoch 4/50
54/54 [==============================] - 0s 2ms/step - loss: 0.3031 - accuracy: 0.8784 - val_loss: 0.2823 - val_accuracy: 0.8930
Epoch 5/50
54/54 [==============================] - 0s 1ms/step - loss: 0.2968 - accuracy: 0.8941 - val_loss: 0.2763 - val_accuracy: 0.8977
Epoch 6/50
54/54 [==============================] - 0s 2ms/step - loss: 0.2893 - accuracy: 0.8982 - val_loss: 0.2705 - val_accuracy: 0.8744
Epoch 7/50
54/54 [==============================] - 0s 1ms/step - loss: 0.2832 - accuracy: 0.9023 - val_loss: 0.2636 - val_accuracy: 0.8977
Epoch 8/50
54/54 [==

In [53]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=64, input_dim=5))
nn_model.add(tf.keras.layers.Dense(units=64))
nn_model.add(tf.keras.layers.Dense(units=64))
nn_model.add(tf.keras.layers.Dense(units=64))

nn_model.add(PReLU())  # Adding PReLU activation function here
# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="RMSprop", metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Epoch 1/50
54/54 [==============================] - 1s 4ms/step - loss: 0.3686 - accuracy: 0.8493 - val_loss: 0.2979 - val_accuracy: 0.8884
Epoch 2/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3219 - accuracy: 0.8802 - val_loss: 0.3250 - val_accuracy: 0.8512
Epoch 3/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3147 - accuracy: 0.8831 - val_loss: 0.2878 - val_accuracy: 0.8884
Epoch 4/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3070 - accuracy: 0.8871 - val_loss: 0.2858 - val_accuracy: 0.8884
Epoch 5/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3011 - accuracy: 0.8901 - val_loss: 0.2762 - val_accuracy: 0.9070
Epoch 6/50
54/54 [==============================] - 0s 1ms/step - loss: 0.2952 - accuracy: 0.8877 - val_loss: 0.3009 - val_accuracy: 0.8744
Epoch 7/50
54/54 [==============================] - 0s 2ms/step - loss: 0.2915 - accuracy: 0.8906 - val_loss: 0.2811 - val_accuracy: 0.8977
Epoch 8/50
54/54 [==

All modifications to learning speed had decreased accuracy for both testing and validation -- keep original settings. Do not specify. 

In [54]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=64, input_dim=5))
nn_model.add(tf.keras.layers.Dense(units=64))
nn_model.add(tf.keras.layers.Dense(units=64))

nn_model.add(PReLU())  # Adding PReLU activation function here
# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.005)

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Epoch 1/50
54/54 [==============================] - 1s 4ms/step - loss: 0.3777 - accuracy: 0.8429 - val_loss: 0.2884 - val_accuracy: 0.8930
Epoch 2/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3324 - accuracy: 0.8662 - val_loss: 0.2800 - val_accuracy: 0.9116
Epoch 3/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3264 - accuracy: 0.8743 - val_loss: 0.3243 - val_accuracy: 0.8605
Epoch 4/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3144 - accuracy: 0.8854 - val_loss: 0.2975 - val_accuracy: 0.8837
Epoch 5/50
54/54 [==============================] - 0s 2ms/step - loss: 0.3044 - accuracy: 0.8842 - val_loss: 0.2815 - val_accuracy: 0.8977
Epoch 6/50
54/54 [==============================] - 0s 2ms/step - loss: 0.3066 - accuracy: 0.8866 - val_loss: 0.2940 - val_accuracy: 0.8930
Epoch 7/50
54/54 [==============================] - 0s 2ms/step - loss: 0.2852 - accuracy: 0.8941 - val_loss: 0.3391 - val_accuracy: 0.8651
Epoch 8/50
54/54 [==

In [55]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=64, input_dim=5))
nn_model.add(tf.keras.layers.Dense(units=64))
nn_model.add(tf.keras.layers.Dense(units=64))

nn_model.add(PReLU())  # Adding PReLU activation function here
# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Epoch 1/50
54/54 [==============================] - 1s 4ms/step - loss: 0.3828 - accuracy: 0.8464 - val_loss: 0.3075 - val_accuracy: 0.8698
Epoch 2/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3217 - accuracy: 0.8778 - val_loss: 0.3338 - val_accuracy: 0.8419
Epoch 3/50
54/54 [==============================] - 0s 2ms/step - loss: 0.3158 - accuracy: 0.8738 - val_loss: 0.2944 - val_accuracy: 0.8744
Epoch 4/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3033 - accuracy: 0.8860 - val_loss: 0.2923 - val_accuracy: 0.8744
Epoch 5/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3004 - accuracy: 0.8860 - val_loss: 0.2832 - val_accuracy: 0.8791
Epoch 6/50
54/54 [==============================] - 0s 1ms/step - loss: 0.2915 - accuracy: 0.8924 - val_loss: 0.2648 - val_accuracy: 0.8930
Epoch 7/50
54/54 [==============================] - 0s 2ms/step - loss: 0.2915 - accuracy: 0.8947 - val_loss: 0.2618 - val_accuracy: 0.8977
Epoch 8/50
54/54 [==

In [56]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=64, input_dim=5))
nn_model.add(tf.keras.layers.Dense(units=64))
nn_model.add(tf.keras.layers.Dense(units=64))

nn_model.add(PReLU())  # Adding PReLU activation function here
# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Epoch 1/50
54/54 [==============================] - 1s 4ms/step - loss: 0.3980 - accuracy: 0.8202 - val_loss: 0.5333 - val_accuracy: 0.7721
Epoch 2/50
54/54 [==============================] - 0s 2ms/step - loss: 0.3689 - accuracy: 0.8517 - val_loss: 0.2916 - val_accuracy: 0.8698
Epoch 3/50
54/54 [==============================] - 0s 2ms/step - loss: 0.3519 - accuracy: 0.8668 - val_loss: 0.3354 - val_accuracy: 0.8558
Epoch 4/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3432 - accuracy: 0.8633 - val_loss: 0.2951 - val_accuracy: 0.8651
Epoch 5/50
54/54 [==============================] - 0s 2ms/step - loss: 0.3296 - accuracy: 0.8767 - val_loss: 0.3232 - val_accuracy: 0.8465
Epoch 6/50
54/54 [==============================] - 0s 2ms/step - loss: 0.3220 - accuracy: 0.8802 - val_loss: 0.3047 - val_accuracy: 0.8744
Epoch 7/50
54/54 [==============================] - 0s 2ms/step - loss: 0.3172 - accuracy: 0.8773 - val_loss: 0.3330 - val_accuracy: 0.8884
Epoch 8/50
54/54 [==

In [57]:
# Calculate Spearman's rank correlation coefficients with the target variable
spearman_corr = df.corr(method='spearman')['Diagnosis'].abs().sort_values(ascending=False)

# Print the ranked features based on Spearman's correlation coefficients
print("Ranked Features based on Spearman's Correlation Coefficients:")
print(spearman_corr)

# Select the top 'k' features based on your criteria
# For example, selecting the top 5 features
top_features = spearman_corr.index[:10].tolist()
print("Top Features selected:")
print(top_features)

Ranked Features based on Spearman's Correlation Coefficients:
Diagnosis                    1.000000
FunctionalAssessment         0.366687
ADL                          0.330450
MemoryComplaints             0.306742
MMSE                         0.236271
BehavioralProblems           0.224350
SleepQuality                 0.056069
EducationLevel               0.043325
CholesterolHDL               0.042542
Hypertension                 0.035080
FamilyHistoryAlzheimers      0.032900
CholesterolLDL               0.032010
Diabetes                     0.031508
CardiovascularDisease        0.031490
BMI                          0.026402
Disorientation               0.024648
CholesterolTriglycerides     0.023072
HeadInjury                   0.021411
Gender                       0.020975
PersonalityChanges           0.020627
Confusion                    0.019186
Ethnicity                    0.017744
SystolicBP                   0.015822
DifficultyCompletingTasks    0.009069
DietQuality               

In [58]:
# Define the deep learning model 
final5_model = tf.keras.models.Sequential()
final5_model.add(tf.keras.layers.Dense(units=64, input_dim=5))
final5_model.add(tf.keras.layers.Dense(units=64))
final5_model.add(tf.keras.layers.Dense(units=64))

final5_model.add(PReLU())  # Adding PReLU activation function here
# Output layer/binary classification so 1 neuron for 0 or 1
final5_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
final5_model.compile(loss="binary_crossentropy", optimizer="RMSprop", metrics=["accuracy"])

# Train the model w/ validation dataset
fit5_model = final5_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = final5_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = final5_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Epoch 1/50
54/54 [==============================] - 1s 4ms/step - loss: 0.3811 - accuracy: 0.8511 - val_loss: 0.3145 - val_accuracy: 0.8558
Epoch 2/50
54/54 [==============================] - 0s 2ms/step - loss: 0.3193 - accuracy: 0.8796 - val_loss: 0.3218 - val_accuracy: 0.8419
Epoch 3/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3077 - accuracy: 0.8819 - val_loss: 0.2889 - val_accuracy: 0.8651
Epoch 4/50
54/54 [==============================] - 0s 2ms/step - loss: 0.2997 - accuracy: 0.8889 - val_loss: 0.2794 - val_accuracy: 0.8791
Epoch 5/50
54/54 [==============================] - 0s 2ms/step - loss: 0.2944 - accuracy: 0.8866 - val_loss: 0.2742 - val_accuracy: 0.8884
Epoch 6/50
54/54 [==============================] - 0s 1ms/step - loss: 0.2854 - accuracy: 0.8965 - val_loss: 0.2819 - val_accuracy: 0.8930
Epoch 7/50
54/54 [==============================] - 0s 2ms/step - loss: 0.2828 - accuracy: 0.9029 - val_loss: 0.2545 - val_accuracy: 0.9070
Epoch 8/50
54/54 [==

In [59]:
final5_model.save("NN_top5Features_model.h5")

In [60]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.91      0.90       139
           1       0.83      0.82      0.82        76

    accuracy                           0.87       215
   macro avg       0.86      0.86      0.86       215
weighted avg       0.87      0.87      0.87       215



Use random forest classifier model on top 5 features (to compare to full dataset analysis)

In [61]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Create a Random Forest classifier instance
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

# Fit the model with the training data
rf_model.fit(X_train, y_train)

# Predict on the validation set
predictions = rf_model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, predictions)
conf_matrix = confusion_matrix(y_val, predictions)
class_report = classification_report(y_val, predictions)

# Print or use the evaluation metrics as needed
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Accuracy: 0.9488372093023256
Confusion Matrix:
 [[135   3]
 [  8  69]]
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.98      0.96       138
           1       0.96      0.90      0.93        77

    accuracy                           0.95       215
   macro avg       0.95      0.94      0.94       215
weighted avg       0.95      0.95      0.95       215

